In [ ]:
# gold_feedback_metrics.ipynb
# GOAL 1: Satisfaction score metrics (avg, counts, comment stats) by specialty & source
# GOAL 2: Tag usage and average satisfaction per tag
# GOAL 3: Encounter–feedback match rate
# SOURCE: kardia_silver.silver_feedback_enriched
# TARGET: TABLE - gold_feedback_satisfaction
#         TABLE - gold_feedback_tag_analysis
#         TABLE - gold_feedback_encounter_match
# TRIGGER: Full snapshot overwrite each run — simple for small datasets. */

from kflow.auth_adls import ensure_adls_oauth
from kflow.config import GOLD_DB, GOLD_DIR

# Configure Spark with ADLS OAuth credentials and return base ABFS path
abfss_base = ensure_adls_oauth()

In [0]:
# Ensure Gold DB exists
spark.sql("USE CATALOG hive_metastore")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {GOLD_DB} LOCATION '{GOLD_DIR}'")
spark.sql(f"USE {GOLD_DB}")

In [0]:
%sql
-- Satisfaction metrics
CREATE OR REPLACE TABLE gold_feedback_satisfaction
USING DELTA AS
SELECT
  COALESCE(provider_specialty, 'Unknown') AS provider_specialty,
  COALESCE(provider_id, 'Unknown')        AS provider_id,
  COALESCE(source, 'Unknown')             AS source,
  COUNT(*)                                AS feedback_count,
  ROUND(AVG(satisfaction_score), 2)       AS avg_score
FROM kardia_silver.silver_feedback_enriched
GROUP BY COALESCE(provider_specialty, 'Unknown'),
         COALESCE(provider_id, 'Unknown'),
         COALESCE(source, 'Unknown');

In [0]:
%sql  
-- 3. Tag usage and average satisfaction per tag  
CREATE OR REPLACE TABLE gold_feedback_tag_analysis
USING DELTA AS

WITH exploded_tags AS (
  SELECT
    satisfaction_score,
    EXPLODE(tags) AS tag
  FROM kardia_silver.silver_feedback_enriched
)

SELECT
  tag,
  COUNT(*) AS feedback_count,
  ROUND(AVG(satisfaction_score), 2) AS avg_score
FROM exploded_tags
GROUP BY tag
ORDER BY feedback_count DESC;

In [0]:
%sql  
-- 4. Encounter–feedback match rate (has valid visit_id?)  
CREATE OR REPLACE TABLE gold_feedback_encounter_match
USING DELTA AS

WITH feedback_flag AS (
  SELECT
    CASE
      WHEN visit_id IS NOT NULL AND TRIM(visit_id) <> '' THEN 1
      ELSE 0
    END AS has_valid_visit
  FROM kardia_silver.silver_feedback_enriched
),

summary AS (
  SELECT
    COUNT(*) AS total_feedback_submissions,
    SUM(has_valid_visit) AS matched_to_encounter_count
  FROM feedback_flag
)

SELECT
  total_feedback_submissions,
  matched_to_encounter_count,
  ROUND(
    100.0 * matched_to_encounter_count / total_feedback_submissions,
    2
  ) AS match_rate_percent
FROM summary;

In [0]:
%sql  
-- 5. Preview: satisfaction metrics  
SELECT *  
FROM gold_feedback_satisfaction
ORDER BY provider_specialty, source  
LIMIT 20;

In [0]:
%sql  
-- Preview: tag analysis  
SELECT * FROM gold_feedback_tag_analysis LIMIT 10;

In [0]:
%sql  
-- Preview: encounter–feedback match rate  
SELECT * FROM gold_feedback_encounter_match LIMIT 10;